In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True, validation_size=5000)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
import datetime

def timestamp():
    d = datetime.datetime.now()
    return d.strftime("%Y/%m/%d/%X")

In [3]:
# Parameters
learning_rate = 0.5
training_epochs = 25
batch_size = 100
display_step = 1
pkeep = 0.75
logs_path_train = '/tmp/tensorflow_logs/example/train' + timestamp()
logs_path_val = '/tmp/tensorflow_logs/example/val' + timestamp()

In [4]:
# Placeholders
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
y_ = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Probability of keeping a node during dropout = 1.0 at 
# test time (no dropout)  and 0.75 at training time
keep_prob = tf.placeholder(tf.float32)

# Variables
W1 = tf.Variable(tf.truncated_normal([784, 200], stddev=0.1), name='Weights') 
b1 = tf.Variable(tf.zeros([200]), name='Bias')
W2 = tf.Variable(tf.truncated_normal([200, 100], stddev=0.1), name='Weights')
b2 = tf.Variable(tf.zeros([100]), name='Bias')
W3 = tf.Variable(tf.truncated_normal([100, 60], stddev=0.1), name='Weights')
b3 = tf.Variable(tf.zeros([60]), name='Bias')
W4 = tf.Variable(tf.truncated_normal([60, 30], stddev=0.1), name='Weights')
b4 = tf.Variable(tf.zeros([30]), name='Bias')
W5 = tf.Variable(tf.truncated_normal([30, 10], stddev=0.1), name='Weights')
b5 = tf.Variable(tf.zeros([10]), name='Bias')

In [5]:
# Construct model and encapsulating all ops into scopes, making
# Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    with tf.variable_scope('Dense1'):
        y1 = tf.nn.relu(tf.matmul(x, W1) + b1)
        y1d = tf.nn.dropout(y1, keep_prob)
    y2 = tf.nn.relu(tf.matmul(y1, W2) + b2)
    y2d = tf.nn.dropout(y2, keep_prob)
    y3 = tf.nn.relu(tf.matmul(y2, W3) + b3)
    y3d = tf.nn.dropout(y3, keep_prob)
    y4 = tf.nn.relu(tf.matmul(y3, W4) + b4)
    y4d = tf.nn.dropout(y4, keep_prob)
    # predicted class (logits)
    y = tf.matmul(y4, W5) + b5
    
with tf.name_scope('Loss'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
    
with tf.name_scope('GD'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
    
with tf.name_scope('Accuracy'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [6]:
# Initializing the variables
init = tf.global_variables_initializer()

In [7]:
# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cross_entropy)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [8]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer_train = tf.summary.FileWriter(logs_path_train, graph=tf.get_default_graph())
    summary_writer_val = tf.summary.FileWriter(logs_path_val, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_train_loss = 0.
        avg_val_loss = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
          
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            batch_x_val, batch_y_val = mnist.validation.next_batch(batch_size)

            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            a, c, summary = sess.run([optimizer, cross_entropy, merged_summary_op], 
                                     feed_dict={x: batch_xs, y_: batch_ys, 
                                                keep_prob: pkeep}) 
            
            c_val, summary_val = sess.run([cross_entropy, merged_summary_op], 
                                                    feed_dict={x: batch_x_val, 
                                                               y_: batch_y_val}) 

            # Write logs at every iteration
            summary_writer_train.add_summary(summary, epoch * total_batch + i)
            summary_writer_val.add_summary(summary_val, epoch * total_batch + i)

            # Compute average loss
            avg_train_loss += c / total_batch
            avg_val_loss += c_val / total_batch
            
        # Display logs per epoch step
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1),
                  "train loss", "{:.9f}".format(avg_train_loss),
                  "val loss", "{:.9f}".format(avg_val_loss))

    print("Optimization Finished!")

    # Test model
    # Calculate accuracy    
    total_test_batch = mnist.test.num_examples // batch_size    
    acc_test_lst = []
    for step in range(mnist.test.images.shape[0] // total_test_batch):        
        acc_test = acc.eval(
            {x: mnist.test.images[step * total_test_batch:(step + 1) * total_test_batch, :], 
             y_: mnist.test.labels[step * total_test_batch:(step + 1) * total_test_batch, :]}) 
        acc_test_lst.append(acc_test)
    print("Accuracy:", np.mean(acc_test_lst))    
    print("Accuracy:", acc.eval({x: mnist.test.images, y_: mnist.test.labels}))

    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/")

Epoch: 0001 train loss 0.652461598 val loss 0.630094816
Epoch: 0002 train loss 0.148699215 val loss 0.139448482
Epoch: 0003 train loss 0.098912121 val loss 0.104075240
Epoch: 0004 train loss 0.076964023 val loss 0.099407430
Epoch: 0005 train loss 0.067597776 val loss 0.098150976
Epoch: 0006 train loss 0.050906090 val loss 0.088968713
Epoch: 0007 train loss 0.040171719 val loss 0.088300583
Epoch: 0008 train loss 0.033935658 val loss 0.090620353
Epoch: 0009 train loss 0.025873286 val loss 0.086285626
Epoch: 0010 train loss 0.024036739 val loss 0.095548699
Epoch: 0011 train loss 0.023523512 val loss 0.093903545
Epoch: 0012 train loss 0.018410850 val loss 0.092338703
Epoch: 0013 train loss 0.017628225 val loss 0.099352712
Epoch: 0014 train loss 0.011537100 val loss 0.100468856
Epoch: 0015 train loss 0.016723401 val loss 0.099414934
Epoch: 0016 train loss 0.012784172 val loss 0.107679026
Epoch: 0017 train loss 0.012715905 val loss 0.106181372
Epoch: 0018 train loss 0.009957719 val loss 0.10